In [49]:
import requests
from requests.exceptions import HTTPError
from urllib.parse import urljoin, urlparse, unquote
import os, codecs

headers = {
    'User-Agent': 'WebIRCrawler',
    'From': 'tharinrath.j@ku.th'
}
seed_url = 'https://www.ku.ac.th/th/'
File_Type = ('pdf', 'doc', 'xls', 'xlsx' ,'ppt', 'exe', 'jpg', 'mpg', 'zip', 'png','jpeg')
Web_Type =  ('html', 'htm' , 'php', 'asp')
domain = "ku.ac.th"
page_count = 0


In [50]:
def checkWeb(link):
    for x in Web_Type:
        if x in link:
            return True
    return False

In [51]:
def checkFile(link):
    for x in File_Type:
        if x in link:
            return True
    return False

In [52]:

def get_page(url):
    global headers
    text = ''
    try:
        response = requests.get(url, headers=headers, timeout=2)
        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print('Success!')
        text = response.text
    return text.lower()


In [53]:
def link_parser(cur_url, raw_html):
    urls = [];  
    pattern_start = '<a href="';  pattern_end = '"'
    index = 0;  length = len(raw_html)
    while index < length:
        start = raw_html.find(pattern_start, index)
        if start > 0:
            start = start + len(pattern_start)
            end = raw_html.find(pattern_end, start)
            link = raw_html[start:end]
            if len(link) > 0:
                if link not in urls:
                    url = urljoin(cur_url,link)
                    link = unquote(url)
                    urls.append(link)
            index = end
        else:
            break
    return urls

In [54]:
def dequeue():
    global frontier_q
    current_url = frontier_q[0]
    frontier_q = frontier_q[1:]
    return current_url

In [55]:
def enqueue(links):
    global frontier_q
    for link in links:
        if (domain in urlparse(link).netloc) and (not checkFile(link)) and (link not in visited_q) and (link not in frontier_q) and ('mailto' not in link) and ("download" not in urlparse(link).path):
            frontier_q.append(link)


In [56]:
def save_files(link,raw_html):
    if(len(raw_html) != 0):
        html_link = urlparse(link)
        filename = html_link.path[html_link.path.rfind("/")+1:]
        if checkWeb(filename):
            path =  "html/" + html_link.netloc  +  html_link.path[:html_link.path.rfind("/")]
            abs_path =  path + '/' + filename
        elif filename == "robots.txt":
            path = 'html/' + html_link.netloc
            abs_path = path + "/robots.txt"
        else:
            path =  "html/" + html_link.netloc  +  html_link.path
            abs_path = path + "/dummy"
        
        try:
            os.makedirs(path, 0o755, exist_ok=True)
            f = codecs.open(abs_path, 'w', 'utf-8')
            f.write(raw_html)
            f.close()
        except:
            print(f"error {abs_path}")
            return 0
        return 1
    return 0

In [57]:
frontier_q = [seed_url,"https://kps.ku.ac.th/", "https://www.src.ku.ac.th/", "https://www.csc.ku.ac.th/th/"]
visited_q = []
robots = []
sitemap = []  

In [58]:
while (page_count < 20000):
    cur_url = dequeue()
    cur_url = urlparse(cur_url)
    visited_q.append(cur_url.netloc + cur_url.path)
    raw_html = get_page(cur_url)
    page_count += save_files(cur_url,raw_html)
    robots_txt = get_page(cur_url.scheme + "://" + cur_url.netloc + "/robots.txt")
    if(len(robots_txt) != 0  and robots_txt[0:4]== "user"):
        save_files(cur_url.scheme + "://" + cur_url.netloc + "/robots.txt",robots_txt)
        if( cur_url.netloc  not in robots):
            f = codecs.open("list_robots.txt", 'a', 'utf-8')
            f.write(cur_url.netloc + "\n")
            f.close()
            if "sitemap" in robots_txt:
                print(robots_txt)
                f = codecs.open("list_sitemap.txt", 'a', 'utf-8')
                f.write(cur_url.netloc + "\n")
                f.close()
        robots.append(cur_url.netloc)
    extracted_links = link_parser(cur_url, raw_html)
    enqueue(extracted_links)
    print(page_count) 

Other error occurred: HTTPSConnectionPool(host='www.ku.ac.th', port=443): Max retries exceeded with url: /th/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f684dad3160>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
Other error occurred: HTTPSConnectionPool(host='www.ku.ac.th', port=443): Max retries exceeded with url: /robots.txt (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f684dad32b0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
0
Other error occurred: HTTPSConnectionPool(host='kps.ku.ac.th', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))
Other error occurred: HTTPSConnectionPool(host='kps.ku.ac.th', port=443): Max retries exceeded with url: /robots.txt (Caused by SSLError(SSLError("bad handshake: Error([(